In [33]:
# Some necessary libraries to manipulate or to visualize the results: (we use latex to visualize)
from math import gcd
from tkinter import *
from IPython.display import display, Latex
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
import io

# We aim to create a fraction as an object, and the different manipulations associated.

# First, we define a fraction as an object in Python (OOP approach) and the basic methods.
# It is a primary basis for the complete program in the next 
class fraction:
    def __init__(self, numerator: int, denominator: int):  # We specify the features' types to avoid confusion.
        self.numerator = numerator
        self.denominator = denominator  # This object has naturally two features.

    def inv(self):  # An inverse function
        self.numerator, self.denominator = self.denominator, self.numerator

    def is_int(self):  # Checking whether the fraction is an integer.
        return self.numerator % self.denominator == 0  

    def canonic(self):  # Putting the fraction into the canonical mathematical form.
        gcd_value = gcd(self.numerator, self.denominator)
        self.numerator //= gcd_value
        self.denominator //= gcd_value

    def disp(self):  # A function to display the fraction in LaTeX:
        display(Latex(f'$ \\frac{{{self.numerator}}}{{{self.denominator}}} $'))

# Now we define the functions associated with the fraction object.
def sum_frac(frac1: fraction, frac2: fraction):
    res = fraction(1, 1)  # Initiating the sum.
    res.numerator = frac1.numerator * frac2.denominator + frac2.numerator * frac1.denominator
    res.denominator = frac1.denominator * frac2.denominator
    res.canonic()  # Using the canonical form, previously defined as a method.
    return res
# Same thing for fraction multiplication. 
def mulp_frac(frac1: fraction, frac2: fraction):
    res = fraction(1, 1)
    res.numerator = frac1.numerator * frac2.numerator
    res.denominator = frac1.denominator * frac2.denominator  
    res.canonic()
    return res 

def mulp_frac_seq(A, B):# Now we define, by recursion, the operations on fractions but on a sequence instead of it just being a binary operation.
    if not A or not B:
        return fraction(1,1)
    return mulp_frac(fraction(A[0],B[0]), mulp_frac_seq(A[1:],B[1:])) # Here A is the sequence of the numerators, and B the sequence of the denominators. 
def sum_frac_seq(A,B):
    if not A or not B:  # Base case: if either list is empty, return 0/1
        return fraction(0, 1)
    return sum_frac(fraction(A[0],B[0]), sum_frac_seq(A[:], B[1:]))

In [40]:
# Examples : 
a= fraction(4, 5)
b= fraction(1, 2)
c= fraction(6,2)
k=fraction(8,12)

In [41]:
# Simple vislualization of fractions and the methods:
a.disp()
b.disp()
a.inv()
a.disp()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [42]:
print(c.is_int())
print(a.is_int())
k.disp()
k.canonic()
k.disp()

True
False


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [43]:
# Simple sum and multiplication
sum_frac(a,b).disp()
mulp_frac(a,b).disp()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [44]:
# Sum and multiplication of a sequence of fractions
sum_frac_seq([1,1,1],[2,2,2]).disp()
mulp_frac_seq([1,1,1],[2,2,2]).disp()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [47]:
# Now a more advanced program for the sole purpose to sum and multiply fractions.

# Function for LaTeX to image conversion (had to use chatgpt for this one since garbage collection caused me a lot of problems)
def latex_to_image(latex_code, label):
    # Create a matplotlib figure to render the LaTeX with inverted colors
    fig, ax = plt.subplots(figsize=(2, 1), dpi=200)  # Adjust the figsize and dpi as needed
    ax.set_facecolor('black')  # Set background to black
    ax.text(0.5, 0.5, f'${latex_code}$', fontsize=20, ha='center', va='center', color='white')  # Set text color to white
    ax.axis('off')

    # Save the figure to a buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, facecolor='black')  # Set facecolor to black
    plt.close(fig)
    buf.seek(0)

    # Convert the buffer to an image that Tkinter can handle
    img = Image.open(buf)
    img_tk = ImageTk.PhotoImage(img)

    # Update the label with the new image
    label.config(image=img_tk)
    label.image = img_tk  # Keep a reference to avoid garbage collection


root = Tk()
root.title("Fractions")
root.geometry("1000x700") 
root.configure(bg='#CDB699')

# Create a Canvas and a Scrollbar
canvas = Canvas(root)
scrollbar = Scrollbar(root, orient="vertical", command=canvas.yview)
scrollable_frame = Frame(canvas)

# Configure the scrollable frame and canvas
scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)

canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

# Packing the canvas and scrollbar
canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

# The buttons commands : ( i.e adding the functionality to the sum and multiplication buttons )
def calculate_sum():
    try:
        numerators_text, denominators_text = fractions_num_seq_Button.get(), fractions_den_seq_Button.get()
        numerators = [int(i) for i in numerators_text.split(',')]
        denominators = [int(i) for i in denominators_text.split(',')]
        res = sum_frac_seq(numerators, denominators)
        latex_to_image(f"\\sum_{{i=1}}^{{{len(numerators)}}} f_i = \\frac{{{res.numerator}}}{{{res.denominator}}}", result_label_sum)
    except Exception as e:
        print(f"Error: {e}")
        result_label_sum.config(text="Invalid input or calculation error!")

def calculate_multiply():
    try:
        numerators_text, denominators_text = fractions_num_seq_Button.get(), fractions_den_seq_Button.get()
        numerators = [int(i) for i in numerators_text.split(',')]
        denominators = [int(i) for i in denominators_text.split(',')]
        res = mulp_frac_seq(numerators, denominators)
        latex_to_image(f"\\prod_{{i=1}}^{{{len(numerators)}}} f_i = \\frac{{{res.numerator}}}{{{res.denominator}}}", result_label_sum)
    except Exception as e:
        print(f"Error: {e}")
        result_label_product.config(text="Invalid input or calculation error!")

# The first few Labels and their configuration: 
Label1 = Label(scrollable_frame, text="The following simple program, manipulates fractions.", bg="#BB6464")
Label1.pack(pady=10, padx=5)

image = Image.open("frac.png")  
img = ImageTk.PhotoImage(image)
img_label = Label(scrollable_frame, image=img)
img_label.image = img  
img_label.pack(pady=10)

Label2 = Label(scrollable_frame, text="Please enter the numerators then the denominators of the fractions in question", bg="#BB6464")
Label2.pack(pady=10, padx=5)

image = Image.open("fi.png")  
img = ImageTk.PhotoImage(image)
img_label = Label(scrollable_frame, image=img)
img_label.image = img  
img_label.pack(pady=10)

# Entries (input as a sequence of numerators and denominators)
fractions_num_seq_Button = Entry(scrollable_frame, bg="#C8F2EF")
fractions_num_seq_Button.pack(pady=10, padx=5)

fractions_den_seq_Button = Entry(scrollable_frame, bg="#C8F2EF")
fractions_den_seq_Button.pack(pady=10, padx=5)

multp_Button = Button(scrollable_frame, text="Multiply", bg="#C3DBD9", command=calculate_multiply)
multp_Button.pack(pady=2)

sum_Button = Button(scrollable_frame, text="Sum", bg="#C3DBD9", command=calculate_sum)
sum_Button.pack(pady=2)

# Create labels for displaying results.
result_label_sum = Label(scrollable_frame, bg="#C8F2EF")
result_label_sum.pack(pady=10)

result_label_product = Label(scrollable_frame, bg="#C8F2EF")
result_label_product.pack(pady=10)

# Main window event loop.
root.mainloop()